# Fase 2: Extracción de Detalles de Películas con Selenium

WEBS
- IMDB: https://www.imdb.com/
- Rotten Tomatoes: https://www.rottentomatoes.com/

- Puntuacion de IMDB (en caso de que la tenga). ✅

- Puntuacion de Rotten Tomatoes (Tomatometer). ✅

- Dirección (director/a o directore/as de cada película). ✅

- Gionistas (de cada película). ✅

- Argumento. ✅

- Duración (en minutos). ✅

- Nombre de la película. ✅

1. Importamos librerías

In [1]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.by import By

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

#Imoprtamos librería para ver el % de avance
from tqdm import tqdm

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

2. Importamos csv

In [4]:
# Importamos los datos extraidos de la Fase 1 que estan en formato CSV y os convertimos a DataFrame
Fase1 = pd.read_csv('Fase1.csv')
Fase1

,Tipo,Nombre,Anno_estreno,Mes_estreno,Id_peli,Genero
0,Movie,Multiverso,2024,6,tt10225380,Action
1,Movie,Detour 95,2024,10,tt10844184,Action
2,Movie,The Legend of Johnny Jones,2024,10,tt10928904,Action
3,Movie,Slasher House 3,2024,3,tt11078340,Action
4,Movie,Clank and the Golden Scar,2024,10,tt11374514,Action
...,...,...,...,...,...,...
7267,Short,Pillow,2024,3,tt32314842,Comedy
7268,Short,C*ck Block,2024,5,tt32314940,Comedy
7269,Short,Spot the Potted Plant,2024,4,tt32315880,Comedy
7270,Short,Kodar: The Primordial God of Light and Ether,2024,5,tt32316006,Comedy


3. Creamos zip, que utilizaremos en Rotten Tomatoes

In [6]:
# Extraemos la información de la columna 'Id_peli' y la guardamos en la variable 'lista_id'
lista_id = list(Fase1['Id_peli'])

#Extraemos la información de las columnas 'Nombre' y 'Anno_estrenao' y las guardamos en las variables 'lista_pelis_cortos' y 'annos_pelis_cortos'
lista_nombre_pelis = list(Fase1['Nombre'])
lista_anno_pelis = list(Fase1['Anno_estreno'])
#Unimos esas dos listas en un zip que utilizaremos para buscar dentro de Rotten Tomatoes
zip_pelis_cortos= list(zip(lista_nombre_pelis, lista_anno_pelis))

In [8]:
#Eliminar
mini_lista = lista_id[0:5]

In [9]:
#Eliminar
#Estas serían las listas extraídas de la Fase 1: del csv sacamos una lista con todos los títulos y otra con todos los años
mini_lista_nombre_pelis= lista_nombre_pelis[0:5]
mini_lista_anno_pelis=lista_anno_pelis[0:5]

#Unimos esas dos listas en un zip que utilizaremos para buscar dentro de Rotten Tomatoes
mini_zip_pelis_cortos= list(zip(mini_lista_nombre_pelis, mini_lista_anno_pelis))

In [14]:
# creamos listas para almacenar la información obtetina de las webs
lista_punt_IMDB = []
lista_punt_tomato = []
lista_dire = []
lista_guionistas = []
lista_argumento = []
lista_duracion = []

# lista_nombre pelis -> esta lista ya se ha creado extrayendo información de la Fase1

In [55]:
mini_lista_2=['tt10579986', 'tt10225380', 'tt10272386', 'tt1016301']

---
Código definitivo
---

In [ ]:
# Inicializa el navegador Chrome
driver = webdriver.Chrome()

# Navega a la página web "http://www.imdb.com"
driver.get("http://www.imdb.com")

# Maximizar la ventana del navegador
driver.maximize_window()

# Acepta las cookies haciendo clic en el elemento con el selector CSS
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

#Iniciamos loop for para iterar en la lista de todas las películas
for peli in tqdm(mini_lista_2):

    # Introducimos el número de ID de la peli en el buscador de la web
    driver.find_element('css selector', '#suggestion-search').send_keys(peli, Keys.ENTER)
    sleep(2)

    #Buscamos la puntuación de cada película
    puntuacion=[]
    try:
        #La puntuación está en un botón. Extraemos el texto de ese botón
        boton=driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1)').text
        #Extraemos la puntuación del texto
        parte_punt = boton.split('/')[0] #Esto devuelve un string con la estructura: 'PUNTUACIÓN EN IMDb\n6,6\n'
        #Extraemos la puntuación del string: una secuencia de dígitos seguidos de coma y dígitos
        patron_punt = r'(\d+,\d+)'
        puntuacion_str=re.search(patron_punt, parte_punt).group(0) #Esto devuelve un string '6,6', así que lo pasamos a float
        puntuacion_float=float(puntuacion_str.replace(',', '.'))
        #Appendeamos la puntuación en la lista
        puntuacion.append(puntuacion_float)

    except:
        puntuacion.append(None)

    #Por último, appendeamos la lista final con las puntuaciones
    lista_punt_IMDB.append(puntuacion)

    #Buscamos la información de los directorters de cada película  
    # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
    # un contador i y una lista vacía donde guardar todos los directores de ésta película
    i=0
    directores=[]
    #Iniciamos el loop while
    while True:  
        i+=1
        # Iniciamos try/expect loop para que cuando no tengamos más directores dé error y salganmos del loop while
        try:
            directores.append(driver.find_element('xpath', f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li[{i}]/a').text)
        except:
            break
    #Appendeamos la lista de directores de esta película en la lista general de directores
    if len(directores)!=0:
        lista_dire.append(directores)
    else:
        lista_dire.append(None)


    #Buscamos la información de los guionistas de cada película  
    # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
    # un contador i y una lista vacía donde guardar todos los guionistas de ésta película
    i=0
    guionistas=[]
    #Iniciamos el loop while
    while True:  
        i+=1
        # Iniciamos try/expect loop para que cuando no tengamos más guionistas dé error y salganmos del loop while
        try:
            guionistas.append(driver.find_element('xpath', f'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul/li[{i}]/a').text)
        except:
            break
    #Appendeamos la lista de guinistas de esta película en la lista general de guionistas
    if len(guionistas)!=0:
        lista_guionistas.append(guionistas)
    else:
        lista_guionistas.append(None)


    #Buscamos el argumento de cada película
    argumento=[]
    #Empezamos un try/except loop per si hay películas que no tienen puntuación. En este caso appendearemos NULL en la lista
    try:
        argumento.append(driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text)
    except:          
        argumento.append(None)
    #Appendeamos la lista final de argumentos
    lista_argumento.append(argumento)


    #Buscamos la duración de cada película en minutos 
    #Creamos lista en la que guardaremos los dígitos de las horas y los minutos
    horas_y_minutos=[]

    # Iniciamos try/expect loop para que busque la sección. Si la encuenta la appendea en seccion_duracion, de lo contrario, appendea None.
    try:
        duracion = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(3)').text
    except:
        duracion = 'NULL'
        
    #De la sección, nos devuelve una lista con todos los campos. Hacemos un regex para extraer las horas y los minutos.
    #Buscamos el patrón "Xh Ymin": un número seguido de 'h', seguido de 0 o más espacios, seguido de otro número y 'min'
    patron = r'(\d+)h\s*(\d+)min|(\d+)h\s*(\d+)m'
    
    #Dentro de cada sección, buscamos por texto, que es cada parte de la sección (título, año, duración)
    #Buscamos el patrón definido dentro de cada parte de la sección, y lo guardamos en la variable 'match'
    match=re.search(patron, duracion)
    if match:
        #Si ha encontrado el patrón, guarda el primer grupo de captura (\d+) en la variable 'horas' y el segundo (\d+) en la variable 'minutos'
        horas=match.group(1)
        minutos=match.group(2)
        horas_y_minutos.append((int(horas), int(minutos)))
    else:
        #Si no existe la duración en la sección, appendeamos None
        horas_y_minutos.append(None)

        
    #Ahora, el contenido de horas_y_minutos es [(2, 2), (1, 30), (1, 47), (1, 44), ('NULL)]. Vamos a pasarlo a minutos.
    #Creamos variable para guardar los minutos, de la que beberá la lista_duracion
    duracion_minutos=[]
    #Iteramos horas_y_minutos
    for tiempo in horas_y_minutos:
        #Si es 'NULL', appendeamos None
        if tiempo == None:
            duracion_minutos.append(None)
        #Se puede dar el caso de que sólo haya un dígito, por ejemplo 2 (2 horas exactas). De ser así, multiplicamos las horas por 60.
        elif len(tiempo)==1:
            duracion_minutos.append(tiempo[0]*60)
        else:
            #Si hay más de un dígito, multiplicamos el primero por 60 y le sumamos los minutos.
            duracion_minutos.append((tiempo[0]*60) + (tiempo[1]))
        
    #Por último, appendeamos la lista final con los minutos totales
    lista_duracion.append(duracion_minutos)  
            
#Cerramos el navegador      
driver.close()

In [13]:
#1. Entramos en la website the 'rotten tomatoes' y aceptamos cookies
driver=webdriver.Chrome()
driver.get('https://www.rottentomatoes.com/')
driver.maximize_window()
driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
sleep(3)

#2. Iteramos el zip para extraer su puntuación
for titulo, año in tqdm(mini_zip_pelis_cortos):

    #Intentamos buscar por el título; si existe entra en el primer resultado; de lo contrario, appenda un mensaje de 'NULL' (ponemos NULL para que las futuras consultas en SQL sean más prácticas)
    try: #bBuscamos la peli por título y año en la barra del buscador
        driver.find_element('css selector', '#header-main > search-algolia > search-algolia-controls > input').send_keys(f'{titulo} {año}', Keys.ENTER)
        sleep(3) #Hacemos click en el primer resultado
        driver.find_element('css selector', '#search-results > search-page-result:nth-child(2) > ul > search-page-media-row:nth-child(1) > a:nth-child(2)').click()
        sleep(3)
        try: #busca el tomatómetro y, si existe, lo appendea en la lista_punt_tomato
            lista_punt_tomato.append(driver.find_element('css selector', '#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button:nth-child(3) > rt-text').text)
        except: #si no tiene tomatómetro, appendea 'NULL'
            lista_punt_tomato.append('NULL')
        sleep(3)
    except: #si la peli no existe devuelve mensaje 'NULL'
        lista_punt_tomato.append('NULL')

#Cerramos el navegador      
driver.close() 

100%|██████████| 5/5 [01:34<00:00, 18.97s/it]


In [11]:
#Creamos un zip con las listas que hemos creado para que nos guarde la infoamción de cada una de las pelis en una tupla
zip_fase2 = zip(lista_punt_IMDB, lista_punt_tomato, lista_dire, lista_guionistas, lista_argumento, lista_duracion, lista_nombre_pelis)
lista_tuplas_fase2 = list(zip_fase2)
len(lista_tuplas_fase2)

0

In [ ]:
# Convertimos la lista de tuplas con la informacion a Data Frame y luego lo exportamos con un fichero formato .csv
df_fase2=pd.DataFrame(lista_tuplas_fase2)
df_fase2.columns = ['Punt_IMDB', 'Tomatometer', 'Direccion', 'Guionistas', 'Argumento','Duracion', 'Nombre']
df_fase2.to_csv("Fase2.csv", index=False)

In [83]:
df_fase2

,Punt_IMDB,Tomatometer,Direccion,Guionistas,Argumento,Duracion,Nombre
0,,90%,[Juanjo Martínez],[Juanjo Martínez],"A drama, which explores unknown earths of the ...",[NULL],Multiverso
1,,90%,[James Twyman],"[Matthew Clark ""Ghost"", Tesha Clark]","Candace leaves northern New Jersey for Miami, ...",[NULL],Detour 95
2,,13%,[James Twyman],"[Matthew Clark ""Ghost"", Tesha Clark]",Set in the housing projects of New Jersey. A y...,[NULL],The Legend of Johnny Jones
3,,NULL,[Mj Dixon],[Mj Dixon],It's out of the frying pan and into the fire f...,[NULL],Slasher House 3
4,,NULL,[Pauli Janhunen Calderón],"[Pauli Janhunen Calderón, David Anghel]",Two missions taking place simultaneously in di...,[NULL],Clank and the Golden Scar


In [91]:
df_fese2.notnull()

NameError: name 'df_fese2' is not defined

---

## Pruebas Isa 🦄

In [1]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
from time import sleep

#importamos librerias de selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.by import By

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

#Imoprtamos librería para ver el % de avance
from tqdm import tqdm

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.
import os
import signal
import sys

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver

In [32]:
#Eliminar
mini_lista_2=['tt10579986', 'tt10225380', 'tt10272386', 'tt1016301']

### Versión final

In [2]:
#Importamos los datos de la Fase 1
fase1=pd.read_csv('Fase1.csv')
fase1

,tipo_pelicula,titulo_pelicula,anno_estreno,mes_estreno,id_pelicula,genero_pelicula
0,Movie,Rodney Cecil: Psycho Hero,2011,4,tt1929361,Comedy
1,Short,Object of Desire,2023,9,tt29009897,Drama
2,Short,Making Magic,2015,NaN,tt3407398,Comedy
3,Short,Significant Other,2018,2,tt8042420,Drama
4,Movie,Hosea: A Story of Hope and Forgiveness,2015,2,tt4209582,Drama
...,...,...,...,...,...,...
995,Short,Article 32,2012,10,tt2439330,Drama
996,Movie,The Mongolian Dream,2012,3,tt1979369,Drama
997,Short,Pop!,2011,7,tt2650758,Comedy
998,Movie,Hakki,2024,4,tt30810790,Drama


In [3]:
# Extraemos la información de la columna 'Id_peli' y la guardamos en la variable 'lista_id'
lista_id = list(fase1['id_pelicula'])

#Extraemos la información de las columnas 'Nombre' y 'Anno_estreno' y las guardamos en las variables 'lista_pelis_cortos' y 'annos_pelis_cortos'
lista_nombre_pelis = list(fase1['titulo_pelicula'])
lista_anno_pelis = list(fase1['anno_estreno'])
#Unimos esas dos listas en un zip que utilizaremos para buscar dentro de Rotten Tomatoes
zip_pelis_cortos= list(zip(lista_nombre_pelis, lista_anno_pelis))

In [4]:
# creamos listas para almacenar la información obtetina de las webs
lista_punt_IMDB = []
lista_punt_tomato = []
lista_dire = []
lista_guionistas = []
lista_argumento = []
lista_duracion = []

In [5]:
# Inicializa el navegador Chrome
driver = webdriver.Chrome()

# Navega a la página web "http://www.imdb.com"
driver.get("http://www.imdb.com")

# Maximizar la ventana del navegador
driver.maximize_window()

# Acepta las cookies haciendo clic en el elemento con el selector CSS
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

#Iniciamos loop for para iterar en la lista de todas las películas
for peli in tqdm(lista_id):

    try:
        # Introducimos el número de ID de la peli en el buscador de la web
        driver.find_element('css selector', '#suggestion-search').send_keys(peli, Keys.ENTER)
        sleep(2)

        #Buscamos la puntuación de cada película
        puntuacion=[]
        try:
            #La puntuación está en un botón. Extraemos el texto de ese botón
            boton=driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1)').text
            #Extraemos la puntuación del texto
            parte_punt = boton.split('/')[0] #Esto devuelve un string con la estructura: 'PUNTUACIÓN EN IMDb\n6,6\n'
            #Extraemos la puntuación del string: una secuencia de dígitos seguidos de coma y dígitos
            patron_punt = r'(\d+,\d+)'
            puntuacion_str=re.search(patron_punt, parte_punt).group(0) #Esto devuelve un string '6,6', así que lo pasamos a float
            puntuacion_float=float(puntuacion_str.replace(',', '.'))
            #Appendeamos la puntuación en la lista
            puntuacion.append(puntuacion_float)

        except:
            puntuacion.append(None)

        #Por último, appendeamos la lista final con las puntuaciones
        lista_punt_IMDB.append(puntuacion)

        #Buscamos la información de los directorters de cada película  
        # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
        # un contador i y una lista vacía donde guardar todos los directores de ésta película
        i=0
        directores=[]
        #Iniciamos el loop while
        while True:  
            i+=1
            # Iniciamos try/expect loop para que cuando no tengamos más directores dé error y salganmos del loop while
            try:
                directores.append(driver.find_element('xpath', f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div/ul/li[{i}]/a').text)
            except:
                break
        #Appendeamos la lista de directores de esta película en la lista general de directores
        if len(directores)!=0:
            lista_dire.append(directores)
        else:
            lista_dire.append(None)


        #Buscamos la información de los guionistas de cada película  
        # Definimos los valores iniciales de nuestras variables para iniciar un loop while que serán:
        # un contador i y una lista vacía donde guardar todos los guionistas de ésta película
        i=0
        guionistas=[]
        #Iniciamos el loop while
        while True:  
            i+=1
            # Iniciamos try/expect loop para que cuando no tengamos más guionistas dé error y salganmos del loop while
            try:
                guionistas.append(driver.find_element('xpath', f'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div/ul/li[{i}]/a').text)
            except:
                break
        #Appendeamos la lista de guinistas de esta película en la lista general de guionistas
        if len(guionistas)!=0:
            lista_guionistas.append(guionistas)
        else:
            lista_guionistas.append(None)


        #Buscamos el argumento de cada película
        argumento=[]
        #Empezamos un try/except loop per si hay películas que no tienen puntuación. En este caso appendearemos NULL en la lista
        try:
            argumento.append(driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text)
        except:          
            argumento.append(None)
        #Appendeamos la lista final de argumentos
        lista_argumento.append(argumento)


        #Buscamos la duración de cada película en minutos 
        #Creamos lista en la que guardaremos los dígitos de las horas y los minutos
        horas_y_minutos=[]

        # Iniciamos try/expect loop para que busque la sección. Si la encuenta la appendea en seccion_duracion, de lo contrario, appendea None.
        try:
            duracion = driver.find_element('css selector', '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(3)').text
        except:
            duracion = 'NULL'
            
        #De la sección, nos devuelve una lista con todos los campos. Hacemos un regex para extraer las horas y los minutos.
        #Buscamos el patrón "Xh Ymin": un número seguido de 'h', seguido de 0 o más espacios, seguido de otro número y 'min'
        patron = r'(\d+)h\s*(\d+)min|(\d+)h\s*(\d+)m'
        
        #Dentro de cada sección, buscamos por texto, que es cada parte de la sección (título, año, duración)
        #Buscamos el patrón definido dentro de cada parte de la sección, y lo guardamos en la variable 'match'
        match=re.search(patron, duracion)
        if match:
            #Si ha encontrado el patrón, guarda el primer grupo de captura (\d+) en la variable 'horas' y el segundo (\d+) en la variable 'minutos'
            horas=match.group(1)
            minutos=match.group(2)
            horas_y_minutos.append((int(horas), int(minutos)))
        else:
            #Si no existe la duración en la sección, appendeamos None
            horas_y_minutos.append(None)

            
        #Ahora, el contenido de horas_y_minutos es [(2, 2), (1, 30), (1, 47), (1, 44), ('NULL)]. Vamos a pasarlo a minutos.
        #Creamos variable para guardar los minutos, de la que beberá la lista_duracion
        duracion_minutos=[]
        #Iteramos horas_y_minutos
        for tiempo in horas_y_minutos:
            #Si es 'NULL', appendeamos None
            if tiempo == None:
                duracion_minutos.append(None)
            #Se puede dar el caso de que sólo haya un dígito, por ejemplo 2 (2 horas exactas). De ser así, multiplicamos las horas por 60.
            elif len(tiempo)==1:
                duracion_minutos.append(tiempo[0]*60)
            else:
                #Si hay más de un dígito, multiplicamos el primero por 60 y le sumamos los minutos.
                duracion_minutos.append((tiempo[0]*60) + (tiempo[1]))
            
        #Por último, appendeamos la lista final con los minutos totales
        lista_duracion.append(duracion_minutos)  

    except:
        pass
            
#Cerramos el navegador      
driver.close()   

100%|██████████| 1000/1000 [04:19<00:00,  3.85it/s]


In [6]:
lista_punt_IMDB

[[5.7],
 [9.0],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [None],
 [7.9],
 [None],
 [None],
 [6.1],
 [None],
 [7.9],
 [None],
 [None],
 [None],
 [6.6],
 [None],
 [None],
 [None],
 [None]]

In [9]:
lista_dire

[None, None, None, None, None, None]

In [18]:
lista_guionistas

[None, None, None, None, None, None]

In [19]:
lista_argumento

[[None], [None], [None], [None], [None], [None]]